In [2]:
import os
from dotenv import load_dotenv
import openai
from dataclasses import dataclass, field
from typing import List
import json
from system_prompts import SYSTEM_PROMPT_REPORT_STRUCTURE, SYSTEM_PROMPT_FIRST_SEARCH, SYSTEM_PROMPT_FIRST_SUMMARY, SYSTEM_PROMPT_REFLECTION
from search_tool import tavily_search

### 加载环境变量和预定义函数及数据结构

In [ ]:
load_dotenv()

@dataclass
class Search:
    url: str = ""
    content: str = ""
    
@dataclass
class Research:
    search_history: List[Search] = field(default_factory=list)
    latest_summary: str = ""
    reflection_iteration: int = 0

@dataclass
class Paragraph:
    title: str = ""
    content: str = ""
    research: Research = field(default_factory=Research)

@dataclass
class State:
    report_title: str = ""
    paragraphs: List[Paragraph] = field(default_factory=list)


def clean_json_tags(json_str):
    return json_str.replace("```json", "").replace("```", "").strip()

def update_state_with_search_results(search_results, idx_paragraph, state):

    for search_result in search_results["results"]:
        search = Search(url=search_result["url"], content=search_result["content"])
        state.paragraphs[idx_paragraph].research.search_history.append(search)
    return state



### 初始化状态

In [4]:
STATE = State()

### 报告大纲智能体

In [ ]:
class ReportStructureAgent:
    def __init__(self):
        self.llm = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    def get_system_prompt(self):        
        output_schema_report_structure = {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "title": {"type": "string"},
                        "content": {"type": "string"}
                    }
                }
            }

        example_output_report_structure = """
```json
[
{
    "title": "智能体的定义",
    "content": "智能体是具有感知、决策、执行和学习能力的自主系统。"
},
{
    "title": "智能体的分类",
    "content": "智能体可以分为以下几类："
}
]
```
"""

        system_prompt = f"""
你是一个深度调研助手。针对一个查询任务，请规划一份报告的结构以及应包含的段落内容。
请确保段落的排列顺序合理。
结构制定完成后，你将获得工具来分别为每个部分进行网页搜索和反思。
请按照以下 JSON 格式定义的格式输出结果：

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema_report_structure, indent=2)}
</OUTPUT JSON SCHEMA>

标题（title）和内容（content）属性将用于后续的深入研究。
请确保输出是一个符合上述 JSON 格式定义的 JSON 对象。
只返回 JSON 对象，不要附加任何解释或额外文本。

<EXAMPLE OUTPUT>
{example_output_report_structure}
</EXAMPLE OUTPUT>
"""

        return system_prompt
        
    def run(self, input):
        response = self.llm.chat.completions.create(
            model="gpt-4o-2024-08-06",
            messages=[
                {"role": "system", "content": self.get_system_prompt()},
                {"role": "user", "content": input}
            ],
            stream=False
        )

        report_structure = json.loads(clean_json_tags(response.choices[0].message.content))

        return report_structure

In [ ]:
research_structure_agent = ReportStructureAgent()

research_input = "请帮我调研一下web3.0的开发"

report_structure = research_structure_agent.run(research_input)
for paragraph in report_structure:
    STATE.paragraphs.append(Paragraph(title=paragraph["title"], content=paragraph["content"]))

print("报告大纲：")
for i, paragraph in enumerate(STATE.paragraphs):
    print(f"第{i}个段落标题: {paragraph.title}") 
    print(f"第{i}个段落内容: {paragraph.content}")

### 搜索智能体

In [21]:
input_json_first_search = {
    "title": STATE.paragraphs[0].title,
    "content": STATE.paragraphs[0].content
}


response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role":"system","content":SYSTEM_PROMPT_FIRST_SEARCH},
        {"role":"user","content":json.dumps(input_json_first_search)}
        ],
    temperature=1
    )

search_interface = response.choices[0].message.content
search_input = json.loads(search_interface)

print(search_input["search_query"])

Web3.0 核心概念详解：去中心化、用户数据主权、智能合约


### 搜索工具

In [24]:
search_keyword = search_input["search_query"]

search_output = tavily_search(search_input["search_query"])

STATE = update_state_with_search_results(search_output, 0, STATE)


for search in STATE.paragraphs[0].research.search_history:
    print(search.url)
    print(search.content)
    break


https://qianfanmarket.baidu.com/article/detail/1069116
去中心化：借助区块链技术，Web3实现了数据的去中心化存储和传输，有效降低了单点故障的风险，并提高了系统的抗攻击能力。 · 用户数据主权：在Web3时代，用户数据不再是各大平台的


### 段落输出智能体

In [ ]:
input_json_search_results = {
    "title": STATE.paragraphs[0].title,
    "content": STATE.paragraphs[0].content,
    "search_query": search_input["search_query"],
    "search_results": [ result["raw_content"] for result in search_output["results"] if result["raw_content"]]
}


response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role":"system","content":SYSTEM_PROMPT_FIRST_SUMMARY},
        {"role":"user","content":json.dumps(input_json_first_search)}
        ],
    temperature=1
    )

paragraph_output = json.loads(response.choices[0].message.content)

print(paragraph_output["paragraph_latest_state"])

### 反思智能体

In [ ]:
input_json_reflection = {
    "title": STATE.paragraphs[0].title,
    "content": STATE.paragraphs[0].content,
    "paragraph_latest_state": paragraph_output["paragraph_latest_state"]
}

response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[
        {"role":"system","content":SYSTEM_PROMPT_REFLECTION},
        {"role":"user","content":json.dumps(input_json_reflection)}
        ],
    temperature=1
    )
    
reflection_output = json.loads(response.choices[0].message.content)

print(reflection_output["search_query"])
